In [2]:
import torch
import torch.nn.functional as F
from   torch.utils.data import Dataset, DataLoader
from   torch import nn

In [3]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 

In [39]:
class ds(Dataset):
    def __init__(self, y, xs):
        self.y, self.xs = y, xs
        # self.y = y
        # self.x = x

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.y[idx], self.xs[idx]        

In [11]:
ys = torch.tensor(ys)[:, None] # add a unit dim
# torch.Tensor()
xs = torch.tensor(xs)

In [14]:
print(xs.grad)

None


In [ ]:
# xs.requires_grad()
# xs.requires_grad_() # _ denotes in place assignment


In [18]:
w = torch.tensor([5.0, 5.0, 5.0])
w.requires_grad_()


tensor([5., 5., 5.], requires_grad=True)

In [20]:
pred = xs @ w

In [28]:
obs = [2.0, 2.0, 2.0, 2.0 ]
obs = torch.tensor(obs)

loss = (obs - pred).pow(2).mean()

In [29]:
loss.backward()

In [30]:
loss

tensor(138.3750, grad_fn=<MeanBackward0>)

In [33]:
w.grad

tensor([37.8750, 28.5000, -2.6250])

In [35]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 

In [36]:
ys = torch.tensor(ys)[:, None] 
xs = torch.tensor(xs)

In [43]:
class ds(Dataset):
    def __init__(self, ys, xs):
        self.ys, self.xs = ys, xs
        # self.ys = ys
        # self.x = x

    def __len__(self):
        return len(self.ys)

    def __getitem__(self, idx):
        return self.ys[idx], self.xs[idx] 


In [46]:
data = ds(ys = ys, xs = xs)
data.xs

tensor([[ 2.0000,  3.0000, -1.0000],
        [ 3.0000, -1.0000,  0.5000],
        [ 0.5000,  1.0000,  1.0000],
        [ 1.0000,  1.0000, -1.0000]])

In [48]:
[e for e in dir(data) if e[0]!='_']

['xs', 'ys']

In [54]:
data_dl = DataLoader(dataset=data, batch_size=2, shuffle=False)

In [55]:
next(iter(data_dl))

[tensor([[ 1.],
         [-1.]]),
 tensor([[ 2.0000,  3.0000, -1.0000],
         [ 3.0000, -1.0000,  0.5000]])]

In [56]:
for i, (yi, xi) in enumerate(data_dl):
    print(f'{i}\t{yi}\t{xi}')

0	tensor([[ 1.],
        [-1.]])	tensor([[ 2.0000,  3.0000, -1.0000],
        [ 3.0000, -1.0000,  0.5000]])
1	tensor([[-1.],
        [ 1.]])	tensor([[ 0.5000,  1.0000,  1.0000],
        [ 1.0000,  1.0000, -1.0000]])


In [59]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 4),
            nn.Tanh(),
            nn.Linear(4, 4),
            nn.Tanh(),
            nn.Linear(4, 1)
        )

    def forward(self, xi):
        pred = self.layers(xi)
        return pred

_ = MLP()
_(xs)

tensor([[0.1692],
        [0.2453],
        [0.2347],
        [0.2470]], grad_fn=<AddmmBackward0>)

In [71]:
class MLP(nn.Module):
    def __init__(self, nin= 3, nouts = [4,4,1]):
        super().__init__()
        tmp =[nin]+nouts
        tmp = sum([[nn.Linear(i, j), nn.Tanh()] for i,j in zip(tmp, tmp[1:])], [])
        
        self.layers = nn.ModuleList(tmp[0:-1] )

    def forward(self, xi):
        for l in self.layers:
            xi = l(xi)
        pred = xi
        return pred

In [70]:
# nin= 3
# nouts = [4,4,1]

# tmp =[nin]+nouts
# tmp = sum([[nn.Linear(i, j), nn.Tanh()] for i,j in zip(tmp, tmp[1:])], [])
# tmp[0:-1]

[Linear(in_features=3, out_features=4, bias=True),
 Tanh(),
 Linear(in_features=4, out_features=4, bias=True),
 Tanh(),
 Linear(in_features=4, out_features=1, bias=True)]

In [73]:
n = MLP(nin = 3, nouts=[4, 4, 1])
n

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): Tanh()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): Tanh()
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [74]:
n(xs)

tensor([[0.4542],
        [0.3543],
        [0.4862],
        [0.3627]], grad_fn=<AddmmBackward0>)

In [75]:
def train_epoch(dataloader, model, loss_fn, optimizer, return_loss):
    model.train()

    for i, (yi, xi) in enumerate(dataloader):
        pred = model(xi)
        loss = loss_fn(pred, yi)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    if return_loss:
        return loss.item()

opt = torch.optim.SGD(n.parameters(), lr=1e-3)
train_epoch(dataloader = data_dl, model = n, loss_fn = nn.MSELoss(), optimizer = opt, return_loss = True)


1.30632483959198

In [77]:

n = MLP(nin = 3, nouts=[4, 4, 1])
opt = torch.optim.SGD(n.parameters(), lr=1e-3)

ys, n(xs)

(tensor([[ 1.],
         [-1.],
         [-1.],
         [ 1.]]),
 tensor([[-0.8499],
         [-0.4579],
         [-0.7651],
         [-0.8587]], grad_fn=<AddmmBackward0>))

In [80]:
for k in range(2000):
    _ = train_epoch(dataloader = data_dl, model = n, loss_fn = nn.MSELoss(), optimizer = opt, return_loss = True)
    print(f'{k}\t{_}')


0	0.6017990708351135
1	0.6000342965126038
2	0.5982679128646851
3	0.596500039100647
4	0.5947310328483582
5	0.59296053647995
6	0.5911888480186462
7	0.5894158482551575
8	0.587641716003418
9	0.5858662724494934
10	0.5840899348258972
11	0.5823124051094055
12	0.5805337429046631
13	0.5787542462348938
14	0.5769738554954529
15	0.575192391872406
16	0.573410153388977
17	0.5716272592544556
18	0.5698434114456177
19	0.5680589079856873
20	0.5662736892700195
21	0.564487874507904
22	0.562701404094696
23	0.5609145164489746
24	0.559127151966095
25	0.5573392510414124
26	0.5555509924888611
27	0.5537624359130859
28	0.5519735813140869
29	0.5501843690872192
30	0.5483949780464172
31	0.5466054677963257
32	0.5448157787322998
33	0.5430260896682739
34	0.5412362813949585
35	0.5394465923309326
36	0.5376569032669067
37	0.5358673334121704
38	0.5340780019760132
39	0.5322887897491455
40	0.5304998755455017
41	0.528711199760437
42	0.5269230008125305
43	0.5251350998878479
44	0.5233476758003235
45	0.521560788154602
46	0.5197

In [81]:
ys, n(xs)

(tensor([[ 1.],
         [-1.],
         [-1.],
         [ 1.]]),
 tensor([[ 1.0445],
         [-1.0973],
         [-0.9060],
         [ 0.9323]], grad_fn=<AddmmBackward0>))